In [1]:
import cv2
import time
import numpy as np

import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

In [2]:
class PinholeCam(object):
    '''PiholeCam
    
    This class is for pinhole camera creating a cv2.VideoCapture() object.
    And can set width and height.
    
    Args:
        serial: The unique flag of pinholeCam.
    '''
    def __init__(self, serial):
        self.serial = serial
    
    def openCam(self, width=1920, height=1080):
        import cv2
        self.pinholeCam = cv2.VideoCapture(self.serial)
        if self.pinholeCam.isOpened():
            self.pinholeCam.set(cv2.CAP_PROP_FRAME_WIDTH, width)
            self.pinholeCam.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
            return self.pinholeCam
        else:
            raise Exception("[Error]: Can not open PinholeCam %s" % self.serial)
            
    def get_oriImg(self):
        ret, img = self.pinholeCam.read()
        if ret:
            return img
    
    def get_rectImg(self, img):
        
        rect_img = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
    
    def set_rect(self, mtx, dist):
        self.mtx = mtx
        self.dist = dist

In [3]:
class PinholeCam_Converter():
    '''The ROS cv_bridge for PinholeCam
    
    Args:
        ori_topic : original image topic to Publisher.
        rect_topic: Rect image topic to Publisher.
    '''
    def __init__(self, ori_topic, rect_topic=None):
        self.ori_pub = rospy.Publisher(ori_topic, Image, queue_size=39)
        if rect_topic:
            self.rect_pub = rospy.Publisher(rect_topic, Image, queue_size=39)
        self.bridge = CvBridge()
        
    def publish(self, ori_img, rect_img=None):
        try:
            ori_msg = self.bridge.cv2_to_imgmsg(ori_img, 'bgr8')
            self.ori_pub.publish(ori_msg)
            if rect_img:
                rect_msg = self.bridge.cv2_to_imgmsg(rect_img, 'bgr8')
                self.rect_pub.publish(rect_msg)
        except CvBridgeError as e:
            rospy.logerr('[Error]: Can not converter Image to msg.')

In [6]:
if __name__ == '__main__':
    rospy.init_node('pinholeCam', anonymous=True)
    rate = rospy.Rate(60)
    cam1 = PinholeCam('/dev/video0')
    cam2 = PinholeCam('/dev/video1')
    cam1.openCam()
    cam2.openCam()
    cam1_conv = PinholeCam_Converter('/cam1/color')
    cam2_conv = PinholeCam_Converter('/cam2/color')
    while cam1.pinholeCam.isOpened() and cam2.pinholeCam.isOpened() and not rospy.is_shutdown():
        t0 = time.time()
        img1 = cam1.get_oriImg()
        img2 = cam2.get_oriImg()
        cam1_conv.publish(img1)
        cam2_conv.publish(img2)
        t1 = time.time()
        print('[Rate]: %5f [Info]: Publishing...' % (1./(t1-t0)))
        rate.sleep()
        
    print('Exiting...')
    cam1.pinholeCam.release()
    cam2.pinholeCam.release()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  # Remove the CWD from sys.path while we load stuff.
